Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto.

In [20]:
import sys
from pyspark.sql import SparkSession

In [21]:
spark = (SparkSession.builder.appName("capitulo3").getOrCreate())

In [22]:
csv_file = "gs://bosonit_spark/archivos/sf-fire-calls.csv"

In [23]:
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

In [6]:
df.show()

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

In [14]:
df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

Cuando se define un schema al definir un campo por ejemplo StructField('Delay', FloatType(), True) ¿qué significa el último parámetro Boolean?

nullable : bool , si el atributo puede ser nulo o no. True puede ser nulo. 

Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?

Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos:

JSON

In [13]:
json_table =  df.write.format("json").save("gs://bosonit_spark/archivos/fire_calls.json")

CSV (dándole otro nombre para evitar sobrescribir el fichero origen)

In [15]:
csv_table =  df.write.format("csv").save("gs://bosonit_spark/archivos/fire_calls.csv")

AVRO

In [25]:
avro_table =  df.write.format("avro").save("gs://bosonit_spark/archivos/fire_calls.avro")

AnalysisException: 'Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of "Apache Avro Data Source Guide".;'

Revisar al guardar los ficheros (p.e. json, csv, etc) el número de ficheros creados, revisar su contenido para comprender (constatar) como se guardan.

¿A qué se debe que hayan más de un fichero?

spark particiona el archivo tanto en memoria como en disco, para mejorar el rendimiento de lectura  

¿Cómo obtener el número de particiones de un DataFrame?

In [37]:
jsondf = spark.read.format("csv").load("gs://bosonit_spark/archivos/fire_calls.csv/*")
print(jsondf.rdd.getNumPartitions())

2


¿Qué formas existen para modificar el número de particiones de un DataFrame?

hacer uso de coalesce() o repartition()

Llevar a cabo el ejemplo modificando el número de particiones a 1 y revisar de nuevo el/los ficheros guardados.

In [39]:
fire_calls2 =  df.coalesce(1).write.format("json").option("header", "true").save("gs://bosonit_spark/archivos/fire_calls2.json")